In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras import layers

In [3]:
data = pd.read_csv("../../Data/clean_data.csv")

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data["Tweet Text"],data["Informativeness"],
test_size=.2,random_state=47,stratify=data["Informativeness"])

In [5]:
bert_preproccessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_model = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [6]:
def get_sentence_embedding(sentence):
    preprocessed_sentence = bert_preproccessor(sentence)
    bert_embedding = bert_model(preprocessed_sentence)
    return bert_embedding["pooled_output"]

### Building a sequential model

In [22]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [33]:
text_input = layers.Input(shape=(),dtype=tf.string,name='tweet')

layer = layers.Dense(32, activation="relu", name="hidden")(get_sentence_embedding(text_input)) 
layer = layers.Dense(1, activation="sigmoid", name="output")(layer)

model = tf.keras.Model(inputs=[text_input], outputs=[layer])

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.fit(X_train, y_train,epochs=5)


In [29]:
from tensorflow.keras import backend
backend.clear_session()